In [3]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session\


In [4]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
import matplotlib.image as mpimg
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
# from tensorflow.keras.applications import DenseNet201 #use too much gpu memory 
# from tensorflow.keras.applications import DenseNet121
# from tensorflow.keras.applications import Xception 
# from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications.efficientnet import EfficientNetB7 #uses too much gpu memory
from tensorflow.keras.applications import ResNet50V2 
import warnings
import glob
warnings.filterwarnings("ignore")

In [5]:
train_csv=pd.read_csv("data/data.csv")
train_csv.head()

,ID,X-Co,Y-Co
0,0.jpg,111.0,110.0
1,1.jpg,99.3,120.0
2,2.jpg,111.5,111.5
3,3.jpg,92.0,148.0
4,4.jpg,151.0,111.0


In [6]:
paths = glob.glob("data/images/*.jpg")
# plt.figure(figsize=(20, 20))
# for i in range(10):
#     cur_img = mpimg.imread(paths[i]) 
#     ax = plt.subplot(7, 7, i + 1)
#     plt.imshow(cur_img.astype("uint8"))
#     plt.axis("off")

In [8]:
train_dataimg = ImageDataGenerator(rescale=1.0/255.0)

In [9]:
train_data = train_dataimg.flow_from_dataframe(train_csv,directory="data/images",subset="training",
                                               x_col="ID",y_col=["X-Co","Y-Co"],target_size=(224, 224),batch_size=8,class_mode='raw')


Found 52 validated image filenames.


In [10]:
for image_batch, labels_batch in train_data:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(8, 224, 224, 3)
(8, 2)


In [11]:
def build_model():
    # base_model=DenseNet121(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
    base_model=ResNet50V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
    # base_model=Xception(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
    # base_model=VGG19(weights='imagenet',include_top=False,input_tensor=Input(shape=(224, 224, 3)))
    base_model.trainable = True
    base_model.summary()
    flatten = base_model.output
    flatten = Flatten()(flatten)
    x=Dense(256,activation='relu')(flatten)
    x=Dense(128,activation='relu')(x)
    x=Dense(64,activation='relu')(x)
    x=Dense(32,activation='relu')(x)
    x=Dense(2,activation='linear')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [12]:
model=build_model()

2023-08-31 00:11:45.342733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-31 00:11:45.391700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-31 00:11:45.391927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-31 00:11:45.392278: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

94668760/94668760 [==============================] - 301s 3us/step
Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                      

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=tf.keras.metrics.RootMeanSquaredError())

In [12]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
model.fit(train_data, epochs=500)

Epoch 1/500


2023-08-01 02:29:54.617851: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8903
2023-08-01 02:29:54.712177: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


7/7 [==============================] - 5s 148ms/step - loss: 7919.8667 - root_mean_squared_error: 88.9936
Epoch 2/500
7/7 [==============================] - 1s 95ms/step - loss: 1256.8773 - root_mean_squared_error: 35.4525
Epoch 3/500
7/7 [==============================] - 1s 101ms/step - loss: 1105.6423 - root_mean_squared_error: 33.2512
Epoch 4/500
7/7 [==============================] - 1s 95ms/step - loss: 700.1949 - root_mean_squared_error: 26.4612
Epoch 5/500
7/7 [==============================] - 1s 95ms/step - loss: 541.3923 - root_mean_squared_error: 23.2678
Epoch 6/500
7/7 [==============================] - 1s 102ms/step - loss: 616.8788 - root_mean_squared_error: 24.8370
Epoch 7/500
7/7 [==============================] - 1s 102ms/step - loss: 392.1247 - root_mean_squared_error: 19.8021
Epoch 8/500
7/7 [==============================] - 1s 96ms/step - loss: 197.2219 - root_mean_squared_error: 14.0436
Epoch 9/500
7/7 [==============================] - 1s 95ms/step - loss: 294.6

In [14]:
model.evaluate(train_data)

7/7 [==============================] - 1s 28ms/step - loss: 23.3750 - root_mean_squared_error: 4.8348


[23.374977111816406, 4.8347673416137695]

In [15]:
pred = model.predict(train_data, verbose=1).round(2)

7/7 [==============================] - 1s 27ms/step


In [17]:
model.save("Resnet50V2_try1.h5")